In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
df = pd.read_csv('spam.csv')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
kaam_ka_df = df[['Category','Message']]
x_old = kaam_ka_df['Message']
y_old = kaam_ka_df['Category']

for i in range(0, len(y_old)):
  if(y_old[i]=='ham'):
    y_old[i] = 0
  else:
    y_old[i] = 1

y_old = y_old.astype('int')


In [5]:
x_old,y_old

(0       Go until jurong point, crazy.. Available only ...
 1                           Ok lar... Joking wif u oni...
 2       Free entry in 2 a wkly comp to win FA Cup fina...
 3       U dun say so early hor... U c already then say...
 4       Nah I don't think he goes to usf, he lives aro...
                               ...                        
 5567    This is the 2nd time we have tried 2 contact u...
 5568                 Will ü b going to esplanade fr home?
 5569    Pity, * was in mood for that. So...any other s...
 5570    The guy did some bitching but I acted like i'd...
 5571                           Rofl. Its true to its name
 Name: Message, Length: 5572, dtype: object,
 0       0
 1       0
 2       1
 3       0
 4       0
        ..
 5567    1
 5568    0
 5569    0
 5570    0
 5571    0
 Name: Category, Length: 5572, dtype: int64)

In [6]:
cv = CountVectorizer()
stemer = nltk.stem.PorterStemmer()
tfidf = TfidfVectorizer()


In [7]:
def vactorization(x):
  for i in range (0, len(x)):
    review = re.sub('[^a-zA-Z]', ' ', x[i])
    review = review.lower()
    review = review.split()
    #stemmization and removing stopwords
    review = [stemer.stem(word) for word in review if not word in set(nltk.corpus.stopwords.words('english'))]
    review = ' '.join(review)
    x[i] = review
  print(x)
  x = cv.fit_transform(x).toarray()
  return x
vector_x_old = vactorization(x_old)


0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri wkli comp win fa cup final tkt st m...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    nd time tri contact u u pound prize claim easi...
5568                                b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: Message, Length: 5572, dtype: object


In [8]:
vector_x_old.shape

(5572, 6296)

In [10]:
from google.colab import drive
drive.mount('/content/drive')
from nltk.stem import WordNetLemmatizer
import numpy as np

words = dict()

def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, 'drive/MyDrive/ColabNotebooks/glove.6B.50d.txt')
words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import line_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
from tensorflow.keras.utils import pad_sequences

def message_to_token_list(s):
  tokens = word_tokenize(s)
  lowercased_tokens = [t.lower() for t in tokens]
  lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
  useful_tokens = [t for t in lemmatized_tokens if t in words]
  return useful_tokens

message_to_token_list('@user feet a fathers is apple attention dysfunctional and is')

def lemmatizenall(x):
  for i in range(0, len(x)):
    review = x[i]
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review ]
    x[i] = review
  return x

x_tokens_old = lemmatizenall(x_old)


In [12]:
def message_to_word_vectors(message, word_dict=words):
  vectors = []

  for token in message:
    if token not in word_dict:
      continue

    token_vector = word_dict[token]
    vectors.append(token_vector)

  return np.array(vectors, dtype=float)

In [15]:
maxlen = 0
for i in range(x_old.shape[0]):
  maxlen = max(maxlen, len(x_old[i]))
maxlen

77

In [16]:
from tensorflow.keras.utils import pad_sequences

def glove_implementation(x):
  counter=0;
  for curr in x:
    x[counter]=message_to_word_vectors(curr)
    counter=counter+1

  xx = []

  for i in x:
    xx.append(i)
  xxx = pad_sequences(xx, maxlen = maxlen, padding = 'pre', value = 0.000, dtype = 'float')
  return xxx

x_glove_old = glove_implementation(x_old)

In [17]:
x_glove_old.shape

(5572, 77, 50)

In [19]:
x_glove_old.shape

(5572, 77, 50)

In [20]:
from tensorflow.keras.utils import to_categorical
yy_old = to_categorical(np.array(y_old), num_classes = 3, dtype = 'int')

In [21]:
yy_old.shape, x_glove_old.shape

((5572, 3), (5572, 77, 50))

In [23]:
test_size_old = (int)(yy_old.shape[0]*0.25)

In [24]:

X_test_old = x_glove_old[ 0:test_size_old]
X_train_old =  x_glove_old[ test_size_old:-1]
y_test_old = y_old[ 0:test_size_old]
y_train_old = y_old[test_size_old:-1]

In [26]:
X_train_old.shape, X_test_old.shape, y_train_old.shape, y_test_old.shape

((4178, 77, 50), (1393, 77, 50), (4178,), (1393,))

In [29]:
def flatten_vec(x_glove):
  li = []
  for i in range (x_glove.shape[0]):
    li.append(x_glove[i].flatten())
  return np.array(li)

In [30]:
X_train_old1d = flatten_vec(X_train_old)
X_test_old1d = flatten_vec(X_test_old)

In [32]:
from sklearn import svm

In [33]:
classifier_old = svm.SVC(decision_function_shape='ovo').fit(X_train_old1d, y_train_old)

In [34]:
y_pred_old = classifier_old.predict(X_test_old1d)

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print("******* SVM ********")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old))

classifier_old.score(X_test_old1d, y_test_old)


******* SVM ********
printing old classification report: 
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1191
           1       0.98      0.79      0.87       202

    accuracy                           0.97      1393
   macro avg       0.97      0.89      0.93      1393
weighted avg       0.97      0.97      0.97      1393


confusion matrix for old model: 
[[1188    3]
 [  43  159]]


0.9669777458722182

In [37]:
from sklearn.neighbors import KNeighborsClassifier as knn

In [41]:
knn_classifier_old = knn(n_neighbors=5).fit(X_train_old1d, y_train_old)

In [39]:
y_pred_knn_old = knn_classifier_old.predict(X_test_old1d)

In [40]:
print("**** KNN ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_knn_old))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_knn_old))

knn_classifier_old.score(X_test_old1d, y_test_old)

**** KNN ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1191
           1       0.92      0.23      0.37       202

    accuracy                           0.89      1393
   macro avg       0.90      0.61      0.65      1393
weighted avg       0.89      0.89      0.86      1393


confusion matrix for old model: 
[[1187    4]
 [ 155   47]]


0.8858578607322326

In [42]:
from sklearn.ensemble import RandomForestClassifier

random_forest_old = RandomForestClassifier(random_state=0)
random_forest_old.fit(X_train_old1d, y_train_old)

y_pred_old_random_forest = random_forest_old.predict(X_test_old1d)



print("**** Random forest ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_random_forest))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_random_forest))

random_forest_old.score(X_test_old1d, y_test_old)

**** Random forest ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1191
           1       1.00      0.62      0.77       202

    accuracy                           0.95      1393
   macro avg       0.97      0.81      0.87      1393
weighted avg       0.95      0.95      0.94      1393


confusion matrix for old model: 
[[1191    0]
 [  76  126]]


0.9454414931801867

In [43]:
from sklearn.linear_model import LogisticRegression

LR_old = LogisticRegression(random_state=0).fit(X_train_old1d, y_train_old)

LR_old.fit(X_train_old1d, y_train_old)

y_pred_old_LR = LR_old.predict(X_test_old1d)



print("**** Logistic regression ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_LR))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_LR))

LR_old.score(X_test_old1d, y_test_old)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**** Logistic regression ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1191
           1       0.88      0.84      0.86       202

    accuracy                           0.96      1393
   macro avg       0.93      0.91      0.92      1393
weighted avg       0.96      0.96      0.96      1393


confusion matrix for old model: 
[[1169   22]
 [  33  169]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9605168700646087

In [44]:
from sklearn.naive_bayes import GaussianNB
gnb_old = GaussianNB()
y_pred_old_gnb = gnb_old.fit(X_train_old1d, y_train_old).predict(X_test_old1d)



print("**** Naive Bayse ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_gnb))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_gnb))

gnb_old.score(X_test_old1d, y_test_old)

**** Naive Bayse ****
printing old classification report: 
              precision    recall  f1-score   support

           0       1.00      0.01      0.01      1191
           1       0.15      1.00      0.25       202

    accuracy                           0.15      1393
   macro avg       0.57      0.50      0.13      1393
weighted avg       0.88      0.15      0.05      1393


confusion matrix for old model: 
[[   9 1182]
 [   0  202]]


0.15147164393395549

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gb_old = GradientBoostingClassifier(learning_rate=1.0, random_state=0).fit(X_train_old1d, y_train_old)
y_pred_old_gb = clf_gb_old.predict(X_test_old1d)

In [46]:
print("**** GB ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_gb))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_gb))

clf_gb_old.score(X_test_old1d, y_test_old)

**** GB ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1191
           1       0.85      0.79      0.82       202

    accuracy                           0.95      1393
   macro avg       0.91      0.88      0.90      1393
weighted avg       0.95      0.95      0.95      1393


confusion matrix for old model: 
[[1163   28]
 [  42  160]]


0.949748743718593

In [49]:
from sklearn import tree
tree_old = tree.DecisionTreeClassifier()

y_pred_old_dt = tree_old.fit(X_train_old1d, y_train_old).predict(X_test_old1d)


print("**** Decision Tree ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_dt))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_dt))

tree_old.score(X_test_old1d, y_test_old)


**** Decision Tree ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1191
           1       0.70      0.71      0.70       202

    accuracy                           0.91      1393
   macro avg       0.82      0.83      0.83      1393
weighted avg       0.91      0.91      0.91      1393


confusion matrix for old model: 
[[1128   63]
 [  58  144]]


0.9131371141421393

In [48]:
from sklearn.ensemble import AdaBoostClassifier
clf_old_ab = AdaBoostClassifier(random_state=0)

y_pred_old_ab = clf_old_ab.fit(X_train_old1d, y_train_old).predict(X_test_old1d)


print("**** AdaBoost ****")
print("printing old classification report: ")
print(classification_report(y_test_old, y_pred_old_ab))
print("\nconfusion matrix for old model: ")
print(confusion_matrix(y_test_old, y_pred_old_ab))

clf_old_ab.score(X_test_old1d, y_test_old)

**** AdaBoost ****
printing old classification report: 
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1191
           1       0.84      0.70      0.77       202

    accuracy                           0.94      1393
   macro avg       0.90      0.84      0.86      1393
weighted avg       0.93      0.94      0.94      1393


confusion matrix for old model: 
[[1164   27]
 [  60  142]]


0.9375448671931084